In [17]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import confusion_matrix, accuracy_score
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, SpatialDropout1D
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download("vader_lexicon")


[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\kauak\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [18]:
Tweets = pd.read_csv("Tweets2.csv")
Tweets.shape

(74682, 4)

In [19]:
Tweets.groupby(['sentiment']).size()

sentiment
Irrelevant    12990
Negative      22542
Neutral       18318
Positive      20832
dtype: int64

In [20]:
Tweets.loc[Tweets['sentiment']=='Irrelevant','sentiment'] = 'Neutral'

In [21]:
Tweets.groupby(['sentiment']).size()

sentiment
Negative    22542
Neutral     31308
Positive    20832
dtype: int64

In [22]:
Tweets = Tweets.dropna(subset=['text'])
Tweets.reset_index(drop=True, inplace=True)

# supervisionadoo

In [23]:
token = Tokenizer(num_words=100)
token.fit_on_texts(Tweets['text'].values)

In [24]:
X = token.texts_to_sequences(Tweets['text'].values)
X = pad_sequences(X, padding="post", maxlen=100)

In [25]:
labelencoder = LabelEncoder()
y = labelencoder.fit_transform(Tweets['sentiment'])
print(y)

[2 2 2 ... 2 2 2]


In [34]:
from tensorflow.keras.utils import to_categorical


In [36]:
y = to_categorical(y)

In [37]:
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.4)
X_test


array([[ 2,  7, 19, ...,  0,  0,  0],
       [54,  0,  0, ...,  0,  0,  0],
       [ 2, 14, 28, ...,  0,  0,  0],
       ...,
       [19,  0,  0, ...,  0,  0,  0],
       [ 6,  1, 10, ...,  0,  0,  0],
       [ 2, 22, 51, ...,  0,  0,  0]], dtype=int32)

In [38]:
modelo = Sequential()
modelo.add(Embedding(input_dim= len(token.word_index), output_dim=128, input_length=X.shape[1]))
modelo.add(SpatialDropout1D(0.2))
modelo.add(LSTM(units=196, dropout=0.2, recurrent_dropout=0, activation='tanh',
                recurrent_activation='sigmoid', unroll=False, use_bias=True))
modelo.add(Dense(units=3,activation="softmax"))

In [39]:
modelo.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(modelo.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_1 (Embedding)         │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ spatial_dropout1d_1             │ ?                      │             0 │
│ (SpatialDropout1D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_1 (LSTM)                   │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

None


In [40]:
modelo.fit(X_train, y_train, epochs=5, batch_size=500,verbose=True)

Epoch 1/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 179s 2s/step - accuracy: 0.4115 - loss: 1.0863
Epoch 2/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 181s 2s/step - accuracy: 0.4147 - loss: 1.0845
Epoch 3/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 129s 1s/step - accuracy: 0.4169 - loss: 1.0833
Epoch 4/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 57s 644ms/step - accuracy: 0.4165 - loss: 1.0834
Epoch 5/5
89/89 ━━━━━━━━━━━━━━━━━━━━ 57s 639ms/step - accuracy: 0.4149 - loss: 1.0847


# Vader

In [41]:
mas = SentimentIntensityAnalyzer()
Tweets['vander_sentiment'] = ''

for y in range(len(Tweets.index)):
  x = mas.polarity_scores(Tweets['text'].iloc[y])
  del x['compound']
  maior = max(x,key=x.get) #neg pos neu
  Tweets.loc[y,'vander_sentiment'] = maior

In [43]:
Tweets.groupby(['vander_sentiment']).size()

vander_sentiment
Negative     3660
Neutral     65581
Positive     4755
dtype: int64

In [42]:
Tweets.loc[Tweets['vander_sentiment']== 'neu' , 'vander_sentiment'] = 'Neutral'
Tweets.loc[Tweets['vander_sentiment']== 'neg' , 'vander_sentiment'] = 'Negative'
Tweets.loc[Tweets['vander_sentiment']== 'pos' , 'vander_sentiment'] = 'Positive'

In [44]:
Tweets.groupby(['vander_sentiment']).size()

vander_sentiment
Negative     3660
Neutral     65581
Positive     4755
dtype: int64

In [45]:
y_pred = Tweets['vander_sentiment']
y_test = Tweets['sentiment']
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[ 2004 19902   452]
 [ 1122 28384  1477]
 [  534 17295  2826]]


In [46]:
accuracy = accuracy_score(y_test, y_pred)
print(accuracy)

0.44886210065408944
